### Bob hat einen Freund Namens Charlie. Bob möchte das die Coins welche er von Alice geschickt bekommt NUR ausgegeben werden können wenn Bob und Charlie die Transaktion mit ihren jeweils eigenen privaten Schlüsseln signieren. Alice muss daher die Coins nicht an den öffentlichen Schlüssel von Bob schicken sondern an ein Skript welches die beiden öffentlichen Schlüssel von Bob und Charlie enthält. Diese Art der Transaktion haben wir bereits als **P2MS** oder **Bare Multisig** kennengelernt. P2MS hat jedoch zwei entscheidenden Nachteile.
    1. Je mehr mögliche öffentliche Schlüssel für MultiSig genutzt werden desto länger wird der ScriptPubKey der Lockingtransaktion und damit wird auch die Transaktion selbst immer größere und teurer.
    2. Die Kosten für die extrem große Lockingtransaktion muss der Sender bezahlen. Das ist aber ökonomisch unfair. Wieso soll der Sender den Preis dafür zahlen, dass der Empfänger einen aufwendigeren Mechanismus zur Ausgabe der Coins bekommt.
Die Lösung beider Probleme besteht darin die Coins an den Hash eines Skripts zu schicken. Der Hash ist immer gleich lang. Daher ändert sich für den Sender nicht die Transaktionsgröße je mehr öffentliche SChlüssel dazu kommen. Das Skript und damit auch die größere Datenmenge wird erst beim Verwenden der Coins veröffentlicht und muss daher vom Eigentümer der MultiSig-Transaktion bezahlt werden.
Diese Art der Transaktion wird als **P2SH** bezeichnet. 

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen dreier Nodes und Wallets (Alice, Bob und Charlie)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2PK-Locking Transaktion genutzt wird.
    5. Bob erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen
    6. Charlie erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen

In [1]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2ms_script, p2sh_script, p2pk_script
from src.ecdsa import PrivateKey
from src.crypto import hash160

In [2]:
test = TestShell().setup(num_nodes=3, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]
charlie = test.nodes[2]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)
charlie.createwallet("charlie", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_p2sh_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2sh_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2sh_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_address_1 = bob.getnewaddress("", "legacy")
bob_private_key_wif_format = bob.dumpprivkey(bob_address_1)
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

charlie_address_1 = charlie.getnewaddress("", "legacy")
charlie_private_key_wif_format = charlie.dumpprivkey(charlie_address_1)
charlie_private_key = PrivateKey.convert_wif_format(charlie_private_key_wif_format)
charlie_private_key_int_format = charlie_private_key.get_private_key_int()
charlie_public_key = charlie_private_key.get_public_key()

2024-03-02T18:06:47.203000Z TestFramework (INFO): PRNG seed is: 7119819211273570658
2024-03-02T18:06:47.204000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_5omgsl_o


**1.** Zunächst müssen Bob und Charlie das von ihnen gewünschte Script erzeugen.

In [3]:
bob_and_charlie_skript = p2ms_script([bob_public_key.sec_format(), charlie_public_key.sec_format()], 2, 2)
bob_and_charlie_skript_hash160 = hash160(bytes.fromhex(bob_and_charlie_skript.serialize_script().hex()[2:]))
print(bob_and_charlie_skript_hash160.hex())

1be890f83060ccc07e1de64cbfd63f781fa70b2a


**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an das P2SH Script von Bob und 
versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [4]:
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_previous_tx_id_to_spent = alice_p2sh_funding_tx_parsed["txid"]

for tx_out in alice_p2sh_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + OP_CHECKSIG generieren.

In [5]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2sh_script(bob_and_charlie_skript_hash160)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2SH Lockingtransaktion: 0100000001d439f99c3d52975f276585739dbb3289b8b284fce3b317d7655e63d0be5790040100000000ffffffff01102700000000000017a9141be890f83060ccc07e1de64cbfd63f781fa70b2a87ffffffff


**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [6]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2SH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2ms_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

print(alice_p2ms_locking_tx)

Alice signierte P2SH Lockingtransaktion: 0100000001d439f99c3d52975f276585739dbb3289b8b284fce3b317d7655e63d0be579004010000006a473044022076771f881959b0d47d17e3715767297fa76de9f8a6466c3166b61464b2fa844002204d12f07dbc9d7d4b199b99fd762dedb7bdc4cd17eed0dd4d64c7783774b8f440012102b41352d0c41d74e09eaaddc5f2c04a68ca1bc2336510e553854c79bb1828f727ffffffff01102700000000000017a9141be890f83060ccc07e1de64cbfd63f781fa70b2a87ffffffff
71958714133013f7fb3a74f5b1cf81b13126e2a3985408b0ae23af76e47557fe


**2.** Nun möchten Bob und Charlie die an den Hash ihres Scripts gebundenen Satoshis weiter versenden. Dazu müssen die beiden im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das sie berechtigt sind den Output auszugeben, ihre Signaturen sowie das von ihnen erzeugte Skript vorlegen. Die Reihenfolge in der die Signaturen vorliegen muss die selbe Reihenfolge sein in der die öffentlichen Schlüssel im Skript vorliegen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [7]:
alice_p2ms_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2ms_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2ms_locking_tx_parsed["txid"]

for tx_out in alice_p2ms_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [8]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2MS Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob unsignierte P2MS Spendingtransaktion: 0100000001fe5775e476af23aeb0085498a3e22631b181cfb1f5743afbf7133013148795710000000000ffffffff012823000000000000232102b41352d0c41d74e09eaaddc5f2c04a68ca1bc2336510e553854c79bb1828f727acffffffff


**2.2** Nun können Bob und Charlie diese Transaktion mit ihren privaten Schlüssle signieren und versenden.

In [9]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format, charlie_private_key_int_format], bob_script_sig, 2, 2, bob_and_charlie_skript)
print(bob_and_charlie_skript.serialize_script().hex())

print(f"Bob signierte P2MS Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")
print(bob_spending_transaction.tx_ins[0].script_sig.serialize_script().hex())

bob_p2ms_spending_tx = bob.sendrawtransaction(bob_spending_transaction.serialize_transaction().hex())

#print(bob_p2ms_spending_tx)

TestShell().shutdown()

47522102d054aaa798eed3e99fb8cbf1643cf94b7ad7a51913216aea84d314643a7ae21521022d70d5ec96c38fb61435e860b8f2ef101973aa9a07ae73c30c562480f7cd80da52ae
Bob signierte P2MS Spendingtransaktion: 0100000001fe5775e476af23aeb0085498a3e22631b181cfb1f5743afbf71330131487957100000000da004730440220575b28adc06def67de39f9d34d57fb0c6f59593b0b683b6a719b6c2a7a3115d602205daf83a1188ab3a375c1adcded84c820b6189fc82342e72479a0f01ed09db8770148304502210094865a62634d3684058a4aa1819d7144911fce1dd41e361ee4139d90a0bdcb3502200f72e6b31148e16d48aae4568bc4ec52aec5ceaf0a9766e6ad38eff8238f04210147522102d054aaa798eed3e99fb8cbf1643cf94b7ad7a51913216aea84d314643a7ae21521022d70d5ec96c38fb61435e860b8f2ef101973aa9a07ae73c30c562480f7cd80da52aeffffffff012823000000000000232102b41352d0c41d74e09eaaddc5f2c04a68ca1bc2336510e553854c79bb1828f727acffffffff
da004730440220575b28adc06def67de39f9d34d57fb0c6f59593b0b683b6a719b6c2a7a3115d602205daf83a1188ab3a375c1adcded84c820b6189fc82342e72479a0f01ed09db8770148304502210094865a62634d3684058a4aa1819d

JSONRPCException: mandatory-script-verify-flag-failed (Script evaluated without error but finished with a false/empty top stack element) (-26)

In [10]:
print(bob_and_charlie_skript.serialize_script().hex())
print(bob_and_charlie_skript_hash160.hex())

47522102d054aaa798eed3e99fb8cbf1643cf94b7ad7a51913216aea84d314643a7ae21521022d70d5ec96c38fb61435e860b8f2ef101973aa9a07ae73c30c562480f7cd80da52ae
1be890f83060ccc07e1de64cbfd63f781fa70b2a
